In [ ]:
#!pip install langchain tiktoken pymongo pydantic

In [ ]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()


In [6]:
from datetime import datetime

books_data = [
    {"title": "1984", "author": "George Orwell", "year": 1949, "genre": "Dystopian", "publication_date": datetime(1949, 6, 8)},
    {"title": "To Kill a Mockingbird", "author": "Harper Lee", "year": 1960, "genre": "Drama", "publication_date": datetime(1960, 7, 11)},
    {"title": "Brave New World", "author": "Aldous Huxley", "year": 1932, "genre": "Dystopian", "publication_date": datetime(1932, 1, 1)},
    {"title": "The Great Gatsby", "author": "F. Scott Fitzgerald", "year": 1925, "genre": "Tragedy", "publication_date": datetime(1925, 4, 10)},
    {"title": "The Hobbit", "author": "J.R.R. Tolkien", "year": 1937, "genre": "Fantasy", "publication_date": datetime(1937, 9, 21)}
]

In [11]:
from pydantic import Field, BaseModel,validator
from typing import Optional ,Type
from langchain.tools import BaseTool    

def get_books(start_date: Optional[datetime] = None, end_date: Optional[datetime] = None, genre: Optional[str] = None, author: Optional[str] = None):
    '''
    This function returns a list of books that match the given criteria. In a real example it should go query mongo db or sql with the given parameters and return the results.
    '''
    filtered_books = []
    for book in books_data:
        if start_date and book['publication_date'] < start_date:
            continue
        if end_date and book['publication_date'] > end_date:
            continue
        if genre and book['genre'] != genre:
            continue
        if author and book['author'] != author:
            continue

        filtered_books.append(book)

    return filtered_books

    return 
class GetBooksInput(BaseModel):
    start_date: Optional[datetime] = Field(description=f"start date of time period mentioned in this format YYYY-MM-DD.")
    end_date: Optional[datetime] = Field(description=f"end date of time period mentioned in this format YYYY-MM-DD.")
    genre: Optional[str] = Field(description=f"genre of the book.")
    author: Optional[str] = Field(description=f"author of the book.")

    @validator('start_date', 'end_date', pre=True, allow_reuse=True)
    def parse_date(cls, value):
        if value is None:
            return value
        try:
            return datetime.strptime(value, '%Y-%m-%d')
        except ValueError:
            raise ValueError("Incorrect date format, should be YYYY-MM-DD")
        

class GetBooksTool(BaseTool):
    name = "get_books"
    description = "Use this when you want to get high-level information about books"
    args_schema : Type[BaseModel] = GetBooksInput

    def _run(self, **kwargs):
            
            query_response = get_books(**kwargs)
            return query_response




In [12]:
get_books(genre="Drama")

[{'title': 'To Kill a Mockingbird',
  'author': 'Harper Lee',
  'year': 1960,
  'genre': 'Drama',
  'publication_date': datetime.datetime(1960, 7, 11, 0, 0)}]

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory , ConversationBufferWindowMemory
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.schema.agent import AgentFinish
from langchain.schema.messages import SystemMessage
from langchain.agents import AgentExecutor
from datetime import date

memory = ConversationBufferMemory(memory_key="memory", return_messages=True, output_key="output")
tools = [GetBooksTool()]
llm = ChatOpenAI(temperature=0)
system_message = SystemMessage(content=(f"Today is {date.today()}.You are an agent that helps people find books."))
prompt = OpenAIFunctionsAgent.create_prompt(
                system_message=system_message,
                extra_prompt_messages=[MessagesPlaceholder(variable_name="memory")]
            )
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)


In [10]:
# Open Ai sends the inputs always as string, and pydantic expects the date to be in (datetime) object
#This is just for illustration on how validation can be helpful, validation can also so some string-similary matching or semantic similarity to match stuff to what we have in our database
#Comment the validation from GetBooksInput to see this error
agent_executor.run("find me books published in the 20th century")

ValidationError: 2 validation errors for GetBooksInput
start_date
  invalid datetime format (type=value_error.datetime)
end_date
  invalid datetime format (type=value_error.datetime)

In [17]:

import langchain
langchain.verbose=True
langchain.debug=True

In [18]:
from IPython.display import Markdown , display
display(Markdown(agent_executor.run("find me books published in the 20th century")))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Today is 2023-11-18.You are an agent that helps people find books.\nHuman: find me books published in the 20th century\nAI: Here are some books published in the 20th century:\n\n1. Title: 1984\n   Author: George Orwell\n   Year: 1949\n   Genre: Dystopian\n   Publication Date: June 8, 1949\n\n2. Title: To Kill a Mockingbird\n   Author: Harper Lee\n   Year: 1960\n   Genre: Drama\n   Publication Date: July 11, 1960\n\n3. Title: Brave New World\n   Author: Aldous Huxley\n   Year: 1932\n   Genre: Dystopian\n   Publication Date: January 1, 1932\n\n4. Title: The Great Gatsby\n   Author: F. Scott Fitzgerald\n   Year: 1925\n   Genre: Tragedy\n   Publication Date: April 10, 1925\n\n5. Title: The Hobbit\n   Author: J.R.R. Tolkien\n   Year: 1937\n   Genre: Fantasy\n   Publication Date: September 21, 1937\n\nL

Here are some books published in the 20th century:

1. Title: 1984
   Author: George Orwell
   Year: 1949
   Genre: Dystopian
   Publication Date: June 8, 1949

2. Title: To Kill a Mockingbird
   Author: Harper Lee
   Year: 1960
   Genre: Drama
   Publication Date: July 11, 1960

3. Title: Brave New World
   Author: Aldous Huxley
   Year: 1932
   Genre: Dystopian
   Publication Date: January 1, 1932

4. Title: The Great Gatsby
   Author: F. Scott Fitzgerald
   Year: 1925
   Genre: Tragedy
   Publication Date: April 10, 1925

5. Title: The Hobbit
   Author: J.R.R. Tolkien
   Year: 1937
   Genre: Fantasy
   Publication Date: September 21, 1937

Let me know if you need more information about any of these books.

In [16]:
display(Markdown(agent_executor.run("what about books in the 21st century")))

I'm sorry, but I couldn't find any books published in the 21st century. It's possible that the database I have access to doesn't include recent publications. Is there anything else I can assist you with?

In [19]:
display(Markdown(agent_executor.run("what drama books do we have?")))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Today is 2023-11-18.You are an agent that helps people find books.\nHuman: find me books published in the 20th century\nAI: Here are some books published in the 20th century:\n\n1. Title: 1984\n   Author: George Orwell\n   Year: 1949\n   Genre: Dystopian\n   Publication Date: June 8, 1949\n\n2. Title: To Kill a Mockingbird\n   Author: Harper Lee\n   Year: 1960\n   Genre: Drama\n   Publication Date: July 11, 1960\n\n3. Title: Brave New World\n   Author: Aldous Huxley\n   Year: 1932\n   Genre: Dystopian\n   Publication Date: January 1, 1932\n\n4. Title: The Great Gatsby\n   Author: F. Scott Fitzgerald\n   Year: 1925\n   Genre: Tragedy\n   Publication Date: April 10, 1925\n\n5. Title: The Hobbit\n   Author: J.R.R. Tolkien\n   Year: 1937\n   Genre: Fantasy\n   Publication Date: September 21, 1937\n\nL

Here is a drama book:

1. Title: To Kill a Mockingbird
   Author: Harper Lee
   Year: 1960
   Genre: Drama
   Publication Date: July 11, 1960

Let me know if you need more information about this book or if you would like to explore other genres.

In [20]:
display(Markdown(agent_executor.run("do you have any books by haper lee?")))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Today is 2023-11-18.You are an agent that helps people find books.\nHuman: find me books published in the 20th century\nAI: Here are some books published in the 20th century:\n\n1. Title: 1984\n   Author: George Orwell\n   Year: 1949\n   Genre: Dystopian\n   Publication Date: June 8, 1949\n\n2. Title: To Kill a Mockingbird\n   Author: Harper Lee\n   Year: 1960\n   Genre: Drama\n   Publication Date: July 11, 1960\n\n3. Title: Brave New World\n   Author: Aldous Huxley\n   Year: 1932\n   Genre: Dystopian\n   Publication Date: January 1, 1932\n\n4. Title: The Great Gatsby\n   Author: F. Scott Fitzgerald\n   Year: 1925\n   Genre: Tragedy\n   Publication Date: April 10, 1925\n\n5. Title: The Hobbit\n   Author: J.R.R. Tolkien\n   Year: 1937\n   Genre: Fantasy\n   Publication Date: September 21, 1937\n\nL

Yes, I have a book by Harper Lee:

1. Title: To Kill a Mockingbird
   Author: Harper Lee
   Year: 1960
   Genre: Drama
   Publication Date: July 11, 1960

Let me know if you need more information about this book or if you are interested in other books by Harper Lee.

In [21]:
memory.clear()
display(Markdown(agent_executor.run("do you have any books by haper lee?")))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "do you have any books by haper lee?",
  "memory": []
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Today is 2023-11-18.You are an agent that helps people find books.\nHuman: do you have any books by haper lee?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "get_books",
                "arg

Yes, I have a book by Harper Lee. The book is called "To Kill a Mockingbird" and it was published in 1960. It falls under the genre of Drama.

In [23]:
display(Markdown(agent_executor.run("Show me all the books by jane austen")))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Today is 2023-11-18.You are an agent that helps people find books.\nHuman: do you have any books by haper lee?\nAI: Yes, I have a book by Harper Lee. The book is called \"To Kill a Mockingbird\" and it was published in 1960. It falls under the genre of Drama.\nHuman: Show me all the books by jane austen"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [1.54s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "

I'm sorry, but I couldn't find any books by Jane Austen in my database.

In [31]:
import requests

class GetCurrencyComparisonInput(BaseModel):
    #We can also let our LLM handle the validation but its better to be safe to handle it ourselves but I wanted to show that you can do both
    base_currency: str = Field(..., description="The base currency code, e.g., 'eur' for Euro., Codes should always be lowercased")
    target_currency: str = Field(..., description="The target currency code to compare with, e.g., 'jpy' for Japanese Yen. , Codes should always be lowercased")
    
#If your Tool/Function has simple logic, you can just include the function logic in the _run method  
class GetCurrencyComparisonTool(BaseTool):
    name = "get_currency_comparison"
    description = "Use this tool to compare the value between two currencies"
    args_schema: Type[BaseModel] = GetCurrencyComparisonInput

    def _run(self, base_currency: str, target_currency: str):
        api_url = f"https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@1/latest/currencies/{base_currency}/{target_currency}.json"

        response = requests.get(api_url)
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "Failed to fetch currency data", "status_code": response.status_code, "response": response.text}

# Example Usage
currency_comparison_tool = GetCurrencyComparisonTool()
result = currency_comparison_tool._run(base_currency="eur", target_currency="jpy")
print(result)

{'date': '2023-11-18', 'jpy': 163.21576532}


In [34]:
memory = ConversationBufferWindowMemory(memory_key="memory", return_messages=True, output_key="output",k=3)
tools = [GetBooksTool(), GetCurrencyComparisonTool()]
llm = ChatOpenAI(temperature=0)
system_message = SystemMessage(content=(f"Today is {date.today()}.You are an agent that helps people find books or compare currencies"))
prompt = OpenAIFunctionsAgent.create_prompt(
                system_message=system_message,
                extra_prompt_messages=[MessagesPlaceholder(variable_name="memory")]
            )
agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory)

In [36]:
display(Markdown(agent_executor.run("do you have any books by haper lee?")))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "do you have any books by haper lee?",
  "memory": []
}
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Today is 2023-11-18.You are an agent that helps people find books.\nHuman: do you have any books by haper lee?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [1.11s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "",
            "additional_kwargs": {
              "function_call": {
                "name": "get_books",
               

Yes, we have a book by Harper Lee called "To Kill a Mockingbird". It was published in 1960 and falls under the genre of Drama.

In [37]:
display(Markdown(agent_executor.run("how does egpyitan pounds compare to dollars right now?")))

[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
[inputs]
[llm/start] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Today is 2023-11-18.You are an agent that helps people find books.\nHuman: do you have any books by haper lee?\nAI: Yes, we have a book by Harper Lee called \"To Kill a Mockingbird\". It was published in 1960 and falls under the genre of Drama.\nHuman: how does egpyitan pounds compare to dollars right now?"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:llm:ChatOpenAI] [1.55s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "",
        "generation_info": {
          "finish_reason": "function_call"
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
          

I apologize, but I am currently unable to fetch the currency data to provide you with the current exchange rate between Egyptian pounds (EGP) and US dollars (USD).